Reference: 
1. SpinalNet: Deep Neural Network with Gradual Input
2. https://github.com/dipuk0506/SpinalNet


# 1.0 Read test image 

In [ ]:
import torchvision
from torchvision import transforms
import torch.utils.data as data
from PIL import ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES = True

batch_size_test = 64

TRANSFORM_IMG = transforms.Compose([transforms.Resize((240, 320)), transforms.ToTensor()])

TEST_DATA_PATH = "classification/test/"
test_data = torchvision.datasets.ImageFolder(root=TEST_DATA_PATH, transform=TRANSFORM_IMG)
test_data_loader = data.DataLoader(test_data, batch_size=batch_size_test, shuffle=True)


# 2.0 Test Model

In [ ]:
# classification
# first_HL = 8
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import summary

torch.backends.cudnn.enabled = False

first_HL = 8

class Net(nn.Module):

  def __init__(self):
      super(Net, self).__init__()
      # kernal 5, maxpooling = 2
      self.conv1 = nn.Conv2d(3, 10, kernel_size=5)
      self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
      self.conv2_drop = nn.Dropout2d()
      # features shape = 20*57*77
      self.fc1 = nn.Linear(int((20*57*77)/2), first_HL)  
      self.fc1_1 = nn.Linear(int((20*57*77)/2) + first_HL, first_HL)  
      self.fc1_2 = nn.Linear(int((20*57*77)/2) + first_HL, first_HL)  
      self.fc1_3 = nn.Linear(int((20*57*77)/2) + first_HL, first_HL)  
      self.fc1_4 = nn.Linear(int((20*57*77)/2) + first_HL, first_HL)  
      self.fc1_5 = nn.Linear(int((20*57*77)/2) + first_HL, first_HL) 
      self.fc2 = nn.Linear(first_HL*6, 70)  
        

  def forward(self, x):
    x = F.relu(F.max_pool2d(self.conv1(x), 2))
    x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
    x = x.view(-1, 20*57*77)
    x1 = x[:, 0:int((20*57*77)/2)]
    
    x1 = F.relu(self.fc1(x1))
    x2 = torch.cat([ x[:,int((20*57*77)/2):20*57*77], x1], dim=1)
    x2 = F.relu(self.fc1_1(x2))
    x3 = torch.cat([ x[:,0:int((20*57*77)/2)], x2], dim=1)
    x3 = F.relu(self.fc1_2(x3))
    x4 = torch.cat([ x[:,int((20*57*77)/2):20*57*77], x3], dim=1)
    x4 = F.relu(self.fc1_3(x4))
    x5 = torch.cat([ x[:,0:int((20*57*77)/2)], x4], dim=1)
    x5 = F.relu(self.fc1_4(x5))
    x6 = torch.cat([ x[:,int((20*57*77)/2):20*57*77], x5], dim=1)
    x6 = F.relu(self.fc1_5(x6))

    # before fc2  
    x = torch.cat([x1, x2], dim=1)
    x = torch.cat([x, x3], dim=1)
    x = torch.cat([x, x4], dim=1)
    x = torch.cat([x, x5], dim=1)
    x = torch.cat([x, x6], dim=1)

    # fc2
    x = self.fc2(x)

    return F.log_softmax(x)

In [ ]:
import os
from itertools import chain
import shutil

folder = 'newClassification/model3'
tested_result = pd.DataFrame(columns = ['model', 'epoch', 'rmse'])
model = Net()

for filename in os.listdir(folder):
    print(filename)
    model = torch.load(folder + '/' + filename)
    model.eval()
    predicted = []
    actual = []
    
    for data,target in test_data_loader:
        output = model(data)
        predicted.append(output.data.max(1)[1].cpu().data.numpy())
        actual.append(target.cpu().data.numpy())
        
    loss_rmse = cal_rmse(list(chain.from_iterable(predicted)), list(chain.from_iterable(actual)))
    
    name = filename.split('.')[0]
    name = name.split('_')
    model_name = name[0]
    epoch = name[2]
    print(model_name)  
    print(epoch)  
    
    tested_result.loc[len(tested_result)] = [model_name, epoch, loss_rmse]
    print(tested_result)  
    
    filename = 'tested_result_' + str(model_name) + '_' +str(epoch)+ '.csv'
    tested_result.to_csv(filename, encoding='utf-8', index=False)
    shutil.move(os.getcwd() + '\\' + filename, os.getcwd() + '\\newClassification')

    